In [2]:
ENV["PYTHON"] = raw"C:\Users\xkzmx\anaconda3\python.exe"
using Pkg
Pkg.build("PyCall")

    Building Conda ─→ `C:\Users\xkzmx\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\6e47d11ea2776bc5627421d59cdcc1296c058071\build.log`
    Building PyCall → `C:\Users\xkzmx\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\1fc929f47d7c151c839c5fc1375929766fb8edcc\build.log`
Precompiling project...
  ✓ PyCall
  ✓ ScikitLearn
  2 dependencies successfully precompiled in 20 seconds (325 already precompiled)


In [20]:
using PyCall
pd = pyimport("pandas")
skeLCS = pyimport("skeLCS")
sk1 = pyimport("sklearn.model_selection")
sk2 = pyimport("sklearn.metrics")


PyObject <module 'sklearn.metrics' from 'C:\\Users\\xkzmx\\anaconda3\\lib\\site-packages\\sklearn\\metrics\\__init__.py'>

In [21]:
data = pd.read_csv("nba_logreg.csv")

#Specify the dataset's phenotype label
classLabel = "TARGET_5Yrs"

#Derive the attribute and phenotype array using the phenotype label
dataFeatures = data.drop(classLabel,axis = 1).values
dataPhenotypes = data[classLabel].values

#Optional: Retrieve the headers for each attribute as a length n array
dataHeaders = data.drop(classLabel,axis=1).columns.values

x_train, x_test, y_train, y_test = sk1.train_test_split(
         dataFeatures, dataPhenotypes, random_state=0);

Training model using training data

In [22]:
model = skeLCS.eLCS(learning_iterations = 50000,track_accuracy_while_fit=py"True");
trainedModel = model.fit(x_train,y_train)

PyObject eLCS(learning_iterations=50000, track_accuracy_while_fit=True)

In [23]:
trainScore = trainedModel.score(x_train,y_train)
#print("Final Training Accuracy: "+pystr(trainedModel.score(x_train,y_train)))
@info "Final Training Accuracy: "  trainScore

testScore = trainedModel.score(x_test,y_test)
@info "Final Test Accuracy: "  testScore


┌ Info: Final Training Accuracy: 
│   trainScore = 0.7157544717933038
└ @ Main In[23]:3
┌ Info: Final Test Accuracy: 
│   testScore = 0.6746663717466637
└ @ Main In[23]:6


In [24]:
plot = pyimport("matplotlib.pyplot")
PyCall.fixqtpath()

false

All three plots are displayed in popup windows.

In [28]:
filename = "iterationData.csv"
trainedModel.export_iteration_tracking_data(filename)
dataTracking = pd.read_csv(filename)

iterations = dataTracking["Iteration"].values
accuracy = dataTracking["Accuracy (approx)"].values

plot.xscale("log",base=10) 

plot.plot(iterations,accuracy,label="Accuracy")
plot.xlabel("Iteration")
plot.ylabel("Accuracy")
plot.legend()
plot.show()

sk2.plot_confusion_matrix(model, x_train, y_train)  
plot.show()

sk2.plot_confusion_matrix(model, x_test, y_test)  
plot.show()



Display trained rules

In [39]:
py"""
def printdata(trainedModel,data, filename):
    classLabel = "TARGET_5Yrs"
    trainedModel.export_final_rule_population(data.drop(classLabel,axis=1).columns.values, classLabel,filename,DCAL=False)
"""

In [42]:
py"printdata"(trainedModel,data,"rules.csv")
populationData2 = pd.read_csv("rules.csv")

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs,Fitness,Accuracy,Numerosity,Avg Match Set Size,TimeStamp GA,Iteration Initialized,Specificity,Deletion Probability,Correct Count,Match Count
0,"57.86,106.14",#,"18.925,28.425",#,"10.96,20.84",#,#,#,#,#,#,#,"3.355521318084988,5.260999999999999","6.796000000000001,10.680000000000001",#,#,#,"2.886,4.914",#,1.0,1.0,1.0,2,25.849602,49630,35560,0.315789,0.000525,28,28
1,"57.86,106.14",#,"18.925,28.425","8.120999999999999,12.212499999999999",#,#,#,#,#,#,#,#,"3.355521318084988,5.260999999999999",#,"9.48,18.32","-0.17300000000000004,6.768",#,"2.886,4.914",#,1.0,1.0,1.0,1,25.851078,49630,36007,0.368421,0.000262,27,27
2,"59.28,115.44387410932532",#,"1.9749999999999996,16.825000000000003",#,"1.995,9.405000000000001",#,"0.7707610393777689,1.536",#,"10.500000000000004,71.9",#,#,#,"-1.408,2.408","0.2370000000000001,3.2510000000000003","-0.09600000000000009,2.768","-2.057,5.257","0.13749999999999996,1.0625","-0.3825,0.9824999999999999","-0.3430000000000002,2.5020000000000002",1.0,1.0,1.0,1,77.135103,49967,36752,0.631579,0.000783,252,252
3,"63.185,100.815",#,#,#,#,"38.9,59.957578426489476",#,#,#,#,#,"80.5,119.5",#,#,"4.125261081045,6.888","2.832,8.768",#,#,#,1.0,1.0,1.0,2,60.107377,49870,37074,0.263158,0.001221,77,77
4,"52.125,63.010000000000005",#,#,#,#,#,"-0.5175,0.69",#,#,#,#,#,#,"0.44499999999999984,7.423832166483161","5.164,8.836",#,"0.175,0.825",#,#,0.0,1.0,1.0,2,68.140590,49968,38465,0.263158,0.001384,102,102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,"62.25,115.44387410932532",#,#,#,#,#,"-0.405,1.536",#,#,#,#,"62.099999999999994,89.1","-1.408,2.195",#,"-0.09600000000000009,2.768",#,"0.13749999999999996,2.7375","-0.3825,1.153","2.083,2.5020000000000002",1.0,0.1,1.0,1,78.649979,49993,49993,0.421053,0.000799,0,0
930,#,#,"14.05,24.515963511928007",#,"9.55,19.05",#,"-0.7244999999999999,0.7244999999999999","-1.105,2.8925",#,#,#,#,#,#,#,"2.1370000000000005,3.878",#,"0.11750000000000005,1.6284999999999998",#,1.0,0.1,1.0,1,89.136202,49994,49994,0.315789,0.000905,0,0
931,#,#,#,#,#,"37.26,56.339999999999996",#,"-1.5125000000000002,1.0075","7.100000000000001,57.1",#,#,#,#,#,"7.241589399745579,17.075583085330223","1.122,6.768","0.4375,1.7625000000000002",#,#,1.0,0.1,1.0,1,86.576233,49994,49994,0.315789,0.000879,0,0
932,"51.025,89.91499999999999",#,"0.0625,13.675",#,"1.995,9.405000000000001","34.264,46.136",#,#,#,#,#,#,"0.605,3.9779999999999998","1.2269999999999999,2.963",#,"-2.5919999999999996,4.192",#,"-0.953,0.9824999999999999","-0.3430000000000002,5.117",1.0,1.0,1.0,1,62.000000,49995,49995,0.473684,0.000751,1,1
